In [1]:
# Author: Markus Viljanen

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_tox = pd.read_csv('MIGRATION_data_curation/ECOTOX_MIGRATION_DATASET_2022-11-15.csv')
df_tox.rename(columns={'latin_name':'Species', 'obs_duration_mean_standardized_d': 'Duration_Value', 
                       'conc1_mean_standardized_mgL': 'Value_Value'}, inplace=True)
df_tox = df_tox[['Species', 'SMILES', 'common_name', 'chemical_name', 'Duration_Value', 'Value_Value']]

In [3]:
df_tox.head()

,Species,SMILES,common_name,chemical_name,Duration_Value,Value_Value
0,Morone saxatilis,C=O,Striped Bass,Formaldehyde,4.000000,56.00
1,Lepomis cyanellus,C=O,Green Sunfish,Formaldehyde,4.000000,173.00
2,Cristatella mucedo,C=O,Bryozoan,Formaldehyde,0.187500,1.38
3,Mystus vittatus,C=O,Striped Catfish,Formaldehyde,4.000000,70.00
4,Sphoeroides annulatus,C=O,Bullseye Puffer,Formaldehyde,0.041667,972.00


In [4]:
# Should have been kekulized by the earlier standardization, 21 observations
df_tox.loc[df_tox['SMILES'] == 'CCN(CC)c1cc2oc3cc(ccc3c(c2cc1)C1C=CC=CC=1C(O)=O)N(CC)CC',
          'SMILES'] = 'CCN(CC)C1=CC2=[O+]C3=C(C=CC(=C3)N(CC)CC)C(=C2C=C1)C1=C(C=CC=C1)C(O)=O'

In [5]:
df_tox.shape, df_tox['Species'].unique().shape, df_tox['SMILES'].unique().shape

((52272, 6), (1506,), (2431,))

### Add categorical duration, log10 value

In [6]:
# Categorical Duration feature
temp = df_tox['Duration_Value'].value_counts().sort_index()
breaks = np.unique(np.sort([0.0] + list(temp.index[temp >= 200]) + [df_tox['Duration_Value'].max()]))
df_tox['Duration.Cat'] = pd.cut(df_tox['Duration_Value'], breaks)

In [7]:
temp.index[temp >= 200]

Float64Index([0.0416666666666667, 0.125, 0.25, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0,
              7.0],
             dtype='float64')

In [8]:
df_tox['Duration.Cat'].value_counts().sort_index()

(0.0, 0.0417]        314
(0.0417, 0.125]      618
(0.125, 0.25]        810
(0.25, 0.5]          634
(0.5, 1.0]         14949
(1.0, 2.0]         10762
(2.0, 3.0]          2857
(3.0, 4.0]         20324
(4.0, 5.0]           291
(5.0, 7.0]           713
Name: Duration.Cat, dtype: int64

In [9]:
df_tox['Value_Log10'] = np.log10(df_tox['Value_Value'])

### Get chemical fingerprints

In [10]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem import rdFingerprintGenerator

In [11]:
fingerprints = {}
for s in df_tox['SMILES'].unique():
    m = Chem.MolFromSmiles(s)
    if m is not None:
        arr = np.zeros((0,), dtype=np.int8) #count vector
        DataStructs.ConvertToNumpyArray(AllChem.GetHashedMorganFingerprint(m,2,nBits = 1024),arr)
        fingerprints[s] = arr
    else:
        fingerprints[s] = np.zeros(1024, dtype=np.int8)

In [12]:
fp_columns = ['fp_%d' % i for i in range(1024)]
fingerprints = pd.DataFrame.from_dict(fingerprints, orient='index', columns=fp_columns)
## Convert to log, then scale to [0,1] (standard, log, z-score, z-score log, ...)
#fingerprints = np.log(1+fingerprints)
#fingerprints = fingerprints/fingerprints.max()
# Add SMILES from index
fingerprints = fingerprints.reset_index().rename(columns={'index':'SMILES'})
fingerprints

,SMILES,fp_0,fp_1,fp_2,fp_3,fp_4,fp_5,fp_6,fp_7,fp_8,...,fp_1014,fp_1015,fp_1016,fp_1017,fp_1018,fp_1019,fp_1020,fp_1021,fp_1022,fp_1023
0,C=O,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CC1CC2C3CCC4=CC(=O)C=CC4(C)C3(F)C(O)CC2(C)C1(O...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,1,0,0,0
2,CCC1(C2C=CC=CC=2)C(=O)NC(=O)NC1=O,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,O=P1(NCCCO1)N(CCCl)CCCl,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CC(O)C(O)=O,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,CC(C)C1C2CCC1C1C(=CC=CC=12)N=C(O)C1=CN(C)N=C1C...,0,2,0,1,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
2427,CN1C=C(C(O)=NC2=CC=CC=C2C2=CC(F)=C(F)C(F)=C2)C...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2428,CC(C1C=NC(=CC=1)C(F)(F)F)S(C)(=O)=NC#N,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,CC1C=CC2CC(C)C(NC3N=C(N)N=C(N=3)C(C)F)C=2C=1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0


In [13]:
fingerprints.to_csv('fingerprints.csv', index=False)

In [14]:
# Check that all chemicals have matches
print("%d / %d" % (df_tox.shape[0], df_tox.merge(fingerprints, on='SMILES').shape[0]))

52272 / 52272


In [15]:
# Which chemicals are all zero?
allzeros = (fingerprints[fp_columns] == 0).all(axis=1)
allzeros = allzeros[allzeros].index
print(allzeros)

Int64Index([], dtype='int64')


In [16]:
# Which fingerprints are all zero?
allzeros = (fingerprints[fp_columns] == 0).all(axis=0)
allzeros = allzeros[allzeros].index
print(allzeros)

Index([], dtype='object')


In [17]:
# MACCS fingerprint
from rdkit.Chem import MACCSkeys

In [18]:
fingerprints = {}
for s in df_tox['SMILES'].unique():
    m = Chem.MolFromSmiles(s)
    if m is not None:
        arr = np.zeros((0,), dtype=np.int8) #count vector
        DataStructs.ConvertToNumpyArray(MACCSkeys.GenMACCSKeys(m),arr)
        fingerprints[s] = arr
    else:
        fingerprints[s] = np.zeros(167, dtype=np.int8)

In [19]:
fp_columns = ['fp_%d' % i for i in range(167)]
fingerprints = pd.DataFrame.from_dict(fingerprints, orient='index', columns=fp_columns)
## Convert to log, then scale to [0,1] (standard, log, z-score, z-score log, ...)
#fingerprints = np.log(1+fingerprints)
#fingerprints = fingerprints/fingerprints.max()
# Add SMILES from index
fingerprints = fingerprints.reset_index().rename(columns={'index':'SMILES'})
fingerprints

,SMILES,fp_0,fp_1,fp_2,fp_3,fp_4,fp_5,fp_6,fp_7,fp_8,...,fp_157,fp_158,fp_159,fp_160,fp_161,fp_162,fp_163,fp_164,fp_165,fp_166
0,C=O,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,CC1CC2C3CCC4=CC(=O)C=CC4(C)C3(F)C(O)CC2(C)C1(O...,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,1,1,1,0
2,CCC1(C2C=CC=CC=2)C(=O)NC(=O)NC1=O,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
3,O=P1(NCCCO1)N(CCCl)CCCl,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,0,1,1,1,0
4,CC(O)C(O)=O,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,CC(C)C1C2CCC1C1C(=CC=CC=12)N=C(O)C1=CN(C)N=C1C...,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
2427,CN1C=C(C(O)=NC2=CC=CC=C2C2=CC(F)=C(F)C(F)=C2)C...,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
2428,CC(C1C=NC(=CC=1)C(F)(F)F)S(C)(=O)=NC#N,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
2429,CC1C=CC2CC(C)C(NC3N=C(N)N=C(N=3)C(C)F)C=2C=1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0


In [20]:
fingerprints.to_csv('fingerprints2.csv', index=False)

In [21]:
# Check that all chemicals have matches
print("%d / %d" % (df_tox.shape[0], df_tox.merge(fingerprints, on='SMILES').shape[0]))

52272 / 52272


In [22]:
# Which chemicals are all zero?
allzeros = (fingerprints[fp_columns] == 0).all(axis=1)
allzeros = allzeros[allzeros].index
print(allzeros)

Int64Index([], dtype='int64')


In [23]:
# Which fingerprints are all zero?
allzeros = (fingerprints[fp_columns] == 0).all(axis=0)
allzeros = allzeros[allzeros].index
print(allzeros)

Index(['fp_0', 'fp_1', 'fp_2', 'fp_3', 'fp_4', 'fp_5', 'fp_6', 'fp_7', 'fp_9',
       'fp_10', 'fp_12', 'fp_35', 'fp_166'],
      dtype='object')


### Add train/test identifier

In [24]:
# Shuffle data set, it is possible data set order reveals something about species or chemicals...
df_tox = df_tox.sample(frac=1, random_state=42).reset_index(drop=True)

In [25]:
drugs = df_tox['SMILES'].astype('category').cat.codes.values
species = df_tox['Species'].astype('category').cat.codes.values
duration = df_tox['Duration_Value'].astype(int)
experiments = (df_tox['SMILES'].astype('str') + ' X ' + 
               df_tox['Species'].astype('str')).astype('category').cat.codes.values

In [26]:
ncv  = 5

In [27]:
from sklearn.model_selection import GroupKFold

In [28]:
class LeaveGroupsOut:
    
    def __init__(self, n_splits, groups):
        self.cv = GroupKFold(n_splits=n_splits)
        self.groups = groups
        
    def split(self, X, y=None, groups=None):
        for train, test in self.cv.split(X,y, groups=self.groups):
            yield train, test

In [29]:
index = np.arange(df_tox.shape[0])
df_tox['index'] = index

In [30]:
cv = LeaveGroupsOut(n_splits=ncv, groups=experiments).split(df_tox)
train, test = next(cv)
df_tox["setting1_test"] = np.where(np.isin(index, test), True, False)

In [31]:
cv = LeaveGroupsOut(n_splits=ncv, groups=drugs).split(df_tox)
train, test = next(cv)
df_tox["setting2_test"] = np.where(np.isin(index, test), True, False)

### Add cross validation identifier

In [32]:
ncv  = 5

In [33]:
cv = LeaveGroupsOut(n_splits=ncv, groups=experiments)
for i, (train, test) in enumerate(cv.split(df_tox)):
    df_tox["setting1_test{fold}".format(fold=i+1)] = np.where(np.isin(index, test), True, False)

In [34]:
cv = LeaveGroupsOut(n_splits=ncv, groups=drugs)
for i, (train, test) in enumerate(cv.split(df_tox)):
    df_tox["setting2_test{fold}".format(fold=i+1)] = np.where(np.isin(index, test), True, False)

### Save

In [35]:
df_tox.to_csv('df_tox.csv', index=False)

In [36]:
df_tox.shape, df_tox['Species'].unique().shape, df_tox['SMILES'].unique().shape

((52272, 21), (1506,), (2431,))

In [37]:
1506*2431

3661086